In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random # library for random number generation

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    

import folium # plotting library


In [2]:
CLIENT_ID = 'QK3VHREKNWURY4IVIYPIK1DGHUF4EMTRVG4MPALFODS1BGA4' # your Foursquare ID
CLIENT_SECRET = 'L5JXWRSQMAVQ0DS4OLJGO2UF3ZUBKPUU0YOA3RSFBTC0BRSS' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QK3VHREKNWURY4IVIYPIK1DGHUF4EMTRVG4MPALFODS1BGA4
CLIENT_SECRET:L5JXWRSQMAVQ0DS4OLJGO2UF3ZUBKPUU0YOA3RSFBTC0BRSS


In [3]:
address = 'Colombo,Sri Lanka'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

6.9349969 79.8538463


In [39]:
rest_df=pd.DataFrame()
list_rest = ['Pizza','Mac Donalds\'s','Restaurant','KFC','The Sizzle Colombo','Burger King','Subway','Dinemore']
radius = 5000
results={}
for search_query in list_rest:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results=requests.get(url).json()
    restaurant= results['response']['venues']
    df = json_normalize(restaurant)
    rest_df=rest_df.append(df,ignore_index=True)
    
rest_df.shape

(160, 19)

In [28]:
filtered_columns = ['name', 'categories'] + [col for col in rest_df.columns if col.startswith('location.')] + ['id']
rest_filtered = rest_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
rest_filtered['categories'] = rest_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
rest_filtered.columns = [column.split('.')[-1] for column in rest_filtered.columns]



rest_filtered.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'cc', 'city', 'state', 'country', 'formattedAddress',
       'postalCode', 'neighborhood', 'crossStreet', 'id'],
      dtype='object')

In [6]:
#Cleaning the DataFrame
rest_df_clear=rest_filtered[['name','categories','lat','lng','neighborhood']]
rest_df_clear.head()

,name,categories,lat,lng,neighborhood
0,Pizza Hut,Pizza Place,6.919948,79.858842,NaN
1,Pizza Hut,Pizza Place,6.948814,79.859708,NaN
2,Pizza Hut,Pizza Place,6.919440,79.846916,NaN
3,Domino's Pizza,Pizza Place,6.948319,79.860367,NaN
4,Domino's Pizza,Pizza Place,6.917824,79.863276,NaN


In [7]:
rest_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel


for lat, lng, label in zip(rest_df_clear.lat, rest_df_clear.lng, rest_df_clear.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(rest_map)

rest_map

In [42]:
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(colombo_group)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

ValueError: could not convert string to float: 'Vegetarian / Vegan Restaurant'